## Example script illustrating sound classification on audio stream
This notebook is an example how to use DeGirum PySDK to do sound classification AI inference of an audio stream from local microphone.

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below. 

### Specify where do you want to run your inferences

In [1]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment

### The rest of the cells below should run without any modifications

In [2]:
import degirum as dg # import DeGirum PySDK
import mytools
import sys
import numpy as np
from IPython.display import clear_output

In [3]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

Inference option = 'DeGirum Cloud Platform'


In [4]:
# load YAMNET sound classification model for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v1_yamnet_sound_cls--96x64_quant_n2x_orca_1")

In [ ]:
abort = False # stream abort flag
N = 5 # inference results history depth
history = [] # list of N consecutive inference results

sampling_rate_hz = model.model_info.InputSamplingRate[0]
read_buffer_size = model.model_info.InputWaveformSize[0] // 2 # two read buffers in waveform for half-length overlapping

# Acquire model input stream object
with mytools.open_audio_stream(sampling_rate_hz, read_buffer_size) as stream:
    #
    # AI prediction loop.
    # emit keyboard typing sound to stop
    #
    for res in model.predict_batch(mytools.audio_overlapped_source(stream, lambda: abort)):
        # clear Jupyter output cell
        clear_output(wait = True) 
        
        # add top inference result to history
        history.insert(0, f"{res.results[0]['label']}: {res.results[0]['score']}" )
    
        # keep only N last elements in history
        if len(history) > N:
            history.pop()
    
        # print history
        for m in history:
            print(m)
        
        # check for stop condition
        if res.results[0]['label'] == "Typing":
            abort = True
    